# 프롬프트 엔지니어링 소개
프롬프트 엔지니어링은 자연어 처리 작업을 위해 프롬프트를 설계하고 최적화하는 과정입니다. 적절한 프롬프트를 선택하고, 그 파라미터를 조정하며, 성능을 평가하는 것이 포함됩니다. 프롬프트 엔지니어링은 NLP 모델에서 높은 정확도와 효율성을 달성하는 데 매우 중요합니다. 이 섹션에서는 OpenAI 모델을 활용하여 프롬프트 엔지니어링의 기본 개념을 살펴보겠습니다.


### 연습 1: 토크나이징
OpenAI에서 제공하는 오픈소스 고속 토크나이저 tiktoken을 사용해 토크나이징을 탐구해보세요.
더 많은 예시는 [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst)에서 확인할 수 있습니다.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### 연습 2: Github Models 키 설정 확인

아래 코드를 실행하여 Github Models 엔드포인트가 올바르게 설정되었는지 확인하세요. 이 코드는 간단한 프롬프트를 시도하고 응답이 제대로 오는지 검증합니다. 입력값 `oh say can you see`에 대해 `by the dawn's early light..`와 비슷한 응답이 나오면 정상입니다.


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### 연습 3: 허구의 정보

존재하지 않는 주제나, 사전 학습 데이터셋에 포함되지 않아 LLM이 알지 못할 수 있는(더 최근의) 주제에 대해 프롬프트를 입력했을 때 어떤 결과가 나오는지 살펴보세요. 프롬프트를 다르게 하거나, 다른 모델을 사용했을 때 응답이 어떻게 달라지는지도 확인해 보세요.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### 연습 4: 지시 기반  
"text" 변수를 사용해 주요 내용을 설정하고  
"prompt" 변수를 사용해 그 주요 내용과 관련된 지시를 제공합니다.

여기서는 모델에게 해당 내용을 초등학교 2학년 학생이 이해할 수 있도록 요약해 달라고 요청합니다.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### 연습 5: 복잡한 프롬프트
시스템, 사용자, 어시스턴트 메시지가 모두 포함된 요청을 시도해 보세요.
시스템은 어시스턴트의 컨텍스트를 설정합니다.
사용자와 어시스턴트 메시지는 여러 번의 대화 흐름을 제공합니다.

시스템 컨텍스트에서 어시스턴트의 성격이 "빈정거림"으로 설정된 점에 주목하세요.
다른 성격 컨텍스트를 사용해 보거나, 다른 입력/출력 메시지 시리즈를 시도해 보세요.


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### 연습: 직관 탐색하기
위의 예시들은 새로운 프롬프트(간단한 것, 복잡한 것, 지시문 등)를 만들 때 활용할 수 있는 패턴을 보여줍니다. 우리가 이야기했던 예시, 힌트 등 다양한 아이디어를 탐구할 수 있도록 다른 연습 문제도 직접 만들어 보세요.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확한 내용이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의해 주시기 바랍니다. 중요한 정보의 경우, 전문 번역가에 의한 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
